In [ ]:
import pandas as pd
from NelsonCheck import NelsonCheck as NC

In [ ]:


df = pd.read_csv("all_cities_casesdeaths.csv", parse_dates=['date'])

city_arr = []

cities = [
    "CEBU CITY", 
    "QUEZON CITY",
    "CAVITE"
]
nc_arr = []
for city in cities:

    city_df = df[df['loc']==city].sort_values(by=['date'], ascending=True)
    city_df.insert(0, 'new_id', range(0, len(city_df)))
    city_df['outbreak'] = 0
    # print(city_df)
    cases = city_df.cases.values
    nc = NC.NelsonCheck(data=cases, exclude_rules=[2,4,7,8])
    nc_arr.append(nc)

    for nv in nc.violations:
        # print(nv.rule)
        # print(nv.offenders)
        ids = nv.offenders.index.values
        if(str(nv.rule) == 'Nelson Rule 1'):
            rule = 1
        elif(str(nv.rule) == 'Nelson Rule 3'):
            rule = 1
        elif(str(nv.rule) == 'Nelson Rule 5'):
            rule = 1
        elif(str(nv.rule) == 'Nelson Rule 6'):
            rule = 1   
            
        for this_id in ids:
            city_df.loc[city_df['new_id']==this_id,'outbreak'] = rule
        
    # city_df['outbreak'].fillna(0, inplace=True)
    city_arr.append(city_df)
city_dfs = pd.concat(city_arr)
# city_dfs.to_csv("city_outbreaks_weekly.csv")

In [ ]:
nc_arr[0].plot()

In [ ]:
tagging_df = pd.read_csv("cavite_outbreak_1.csv", parse_dates=['date'])
tagging_df

,loc,cases,deaths,date,outbreak
0,CAVITE,64,0,2016-10-01,0
1,CAVITE,214,1,2016-01-17,0
2,CAVITE,161,2,2016-01-24,0
3,CAVITE,147,1,2016-01-31,0
4,CAVITE,160,0,2016-07-02,0
...,...,...,...,...,...
256,CAVITE,60,0,2020-12-13,0
257,CAVITE,60,0,2020-12-20,0
258,CAVITE,57,0,2020-12-27,0
259,CAVITE,68,0,2021-03-01,0


In [ ]:
loc_df = tagging_df[tagging_df['loc'] == "CAVITE"]

In [ ]:
train = loc_df[loc_df['date'].dt.year < 2019 ]
test = loc_df[loc_df['date'].dt.year == 2019]

In [ ]:
train.shape, test.shape, \
"outbreak_train: " + str(train['outbreak'].sum()) + "/" + str(len(train.index)), \
"outbreak_test: " + str(test['outbreak'].sum()) + "/" + str(len(test.index))


((156, 5), (51, 5), 'outbreak_train: 74/156', 'outbreak_test: 29/51')

In [ ]:
train

,loc,date,cases,deaths,outbreak
0,CAVITE,2016-01-17,214,1,0
1,CAVITE,2016-01-24,161,2,0
2,CAVITE,2016-01-31,147,1,0
3,CAVITE,2016-07-02,160,0,0
4,CAVITE,2016-02-14,150,0,0
...,...,...,...,...,...
151,CAVITE,2018-09-12,238,1,0
152,CAVITE,2018-12-16,245,0,0
153,CAVITE,2018-12-23,141,0,0
154,CAVITE,2018-12-30,163,1,0


In [ ]:
import os
city = "CAVITE"
date = "2018-01-01"
layer = "ndvi"
data_path = "images_1/" + city +  "/" + layer + "/snapshot-" + date + "T00_00_00Z.jpg"
print(data_path)

images_1/CAVITE/ndvi/snapshot-2018-01-01T00_00_00Z.jpg


In [ ]:
from PIL import Image
jpgfile = Image.open(data_path)

In [ ]:
!pip install opencv-python==4.6.0.66
!apt-get update && apt-get install libgl1 -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 43.3 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian buster/main amd64 Packages [7,909 kB]
Get:5 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [358 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [8,788 B]
Fetched 8,489 kB in 1s (6,607 kB/s)




The following additional packages will be installed:
  libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1
  libdrm2 libgl1-mesa-dri libglapi-mesa libglvnd0 libglx-mesa0 libglx0
  libllvm7 libpciaccess0 libsensors-config libsensors5 libx11-xcb1
  libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0 libxcb-present0 lib

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

2022-09-23 11:56:48.224204: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 11:56:48.361337: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-23 11:56:48.366371: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-23 11:56:48.366399: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [ ]:
img = cv2.imread(data_path)
img.shape

(800, 800, 3)

In [ ]:
img_arr=cv2.resize(img,(224,224))
img_arr.shape

(224, 224, 3)

In [ ]:
img_arr

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 52, 194, 154],
        [  0, 153,  82],
        [  0, 153,  81]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 48, 196, 152],
        [  0, 153,  81],
        [  1, 152,  81]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 25, 179, 131],
        [  0, 149,  77],
        [  0, 148,  77]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  4, 149,  82],
        [  0, 159,  92],
        [  0, 158,  94]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0, 129,  62],
        [  8, 180, 118],
        [  7, 177, 125]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0, 128,  62],
        [  5, 178, 122],
        [  1, 175, 130]]

In [ ]:
train

,Unnamed: 0,new_id,loc,cases,deaths,date,outbreak
260,5806,0,QUEZON CITY,14,0.0,2016-01-05,0
261,5791,1,QUEZON CITY,58,0.0,2016-01-17,0
262,5792,2,QUEZON CITY,81,0.0,2016-01-24,0
263,5793,3,QUEZON CITY,85,0.0,2016-01-31,0
264,5838,4,QUEZON CITY,2,167.0,2016-02-10,0
...,...,...,...,...,...,...,...
410,19354,150,QUEZON CITY,253,1.0,2018-11-25,0
411,19339,151,QUEZON CITY,348,2.0,2018-12-08,0
412,19357,152,QUEZON CITY,104,0.0,2018-12-16,0
413,19358,153,QUEZON CITY,176,0.0,2018-12-23,0


In [ ]:
import requests
import wget
import datetime
import os
import numpy

# init vars 
width = 150
height = 150

# year
year = 2013
month = 1
day = 1

diff = 0.1103
start_year = 2013
end_year = 2022

#key: layername, value1: layer, value2: wrap
layers = {
    'fapar' : ['Coastlines_15m,MODIS_Combined_L4_FPAR_4Day,MODIS_Terra_L4_FPAR_8Day,MODIS_Aqua_L4_FPAR_8Day,MODIS_Combined_L4_FPAR_8Day','x,none,none,none,none'],
    # 'edvi' : ['Coastlines_15m,MODIS_Aqua_L3_EVI_16Day','x,none'],
    'ndvi' : ['Coastlines_15m,MODIS_Terra_L3_NDVI_16Day','x,none'],
    # 'lst_m_day' : ['Coastlines_15m,MODIS_Terra_L3_Land_Surface_Temp_Monthly_Day', 'x,none'],
    # 'lst_m_nt' : ['Coastlines_15m,MODIS_Terra_L3_Land_Surface_Temp_Monthly_Night', 'x,none'],
    # 'precip' : ['Coastlines_15m,IMERG_Precipitation_Rate', 'x,none'],
    # 'elev' : ['Coastlines_15m,ASTER_GDEM_Color_Index', 'x,none']
}

cities = {
    'CAVITE': '14.3294,120.9367',
    # 'LAGUNA': '14.2167,121.1667',
    # 'CEBU': '10.3,123.9',
    # 'ILOILO': '10.7167,122.5667',
    # 'QUEZON_CITY': '14.6333,121.0333',
    # 'PANGASINAN': '15.9281,120.3489',
    # 'BULACAN': '14.7928,120.8789',
    # 'NEGROS_OCCIDENTAL': '9.9833,122.8167',
    # 'BATANGAS': '13.75,121.05',
    # 'RIZAL': '14.5842,121.1763',
    # 'PAMPANGA': '15.0333,120.6833',
    # 'BUKIDNON': '7.9,125.0833',
    # 'NUEVA_ECIJA': '15.4833,120.9667',
    # 'TARLAC': '15.4802,120.5979',
    # 'NEGROS ORIENTAL': '9.3103,123.3081',
    # 'BOHOL': '9.65,123.85',
    # 'SOUTH_COTABATO': '6.5,124.85',
    # 'QUEZON': '7.7333,125.1',
    # 'ZAMBOANGA_CITY': '6.9167,122.0833',
}

def latlong_gen(city):
    latlong = cities[city]
    lat_v, long_v = latlong.split(",")
    
    #define size of BBOX?
    LL_lat = round(float(lat_v) - diff,4)
    LL_long = round(float(long_v) - diff,4)
    UR_lat = round(float(lat_v) + diff, 4)
    UR_long = round(float(long_v) + diff, 4)

    bbox = str(LL_lat) + "," + str(LL_long) + "," + str(UR_lat) + "," + str(UR_long) 
    return bbox

def url_gen(full_date_txt, bbox, layers, width, height, wrap):
    url = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=" + \
    full_date_txt + "T00:00:00Z&" + \
    "BBOX=" + bbox + "&CRS=EPSG:4326&" + \
    "LAYERS=" + layers + "&WRAP=" + wrap + "&FORMAT=image/jpeg&" + \
    "WIDTH=" + str(width) + "&HEIGHT=" + str(height) + "&ts=1663663285756"
    return url

for layer_name, layer_wrap in layers.items():
    layer = layer_wrap[0]
    wrap = layer_wrap[1]

    for city, latlong in cities.items():
        directory = city
        parent_dir = "images"
        path = os.path.join(parent_dir, directory, layer_name)
        try:
            os.makedirs(path)
        except:
            pass

        bbox = latlong_gen(city)    
        
        loc_dates = loc_df.date.values

        for loc_date in loc_dates:
            full_date_txt = numpy.datetime_as_string(loc_date, unit='D')
            url = url_gen(full_date_txt, bbox, layer, width, height, wrap)
            file_name = wget.download(url, out=path)
            print('Image Successfully Downloaded: ', file_name)


Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-01-17T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-01-24T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-01-31T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-07-02T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-02-14T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-02-21T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-02-28T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-06-03T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-03-13T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-03-20T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/snapshot-2016-03-27T00_00_00Z.jpg
Image Successfully Downloaded:  images/CAVITE/fapar/sn

In [ ]:
import numpy
numpy.datetime_as_string(qc_df.date.values[0], unit='D')

'2016-01-05'

In [ ]:
import os



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4362acbc-ec93-48fa-977f-7a0e80d02730' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>